In [1]:
import pandas as pd
from sklearn.manifold import TSNE
import xlwings as xw
from sklearn.cluster import DBSCAN
from bioinfokit.visuz import cluster
import plotly.express as px
import math
import numbers
import datetime
from pandas.api.types import is_numeric_dtype

C:\Users\Luke Laptop\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
app = xw.App()
book = xw.Book('DR Metformin Data 8192021 from santi for asutin group.xlsb')
sheet = book.sheets('Master corrected variables')
df = sheet.range('A1').options(pd.DataFrame, expand='table').value
book.close()
app.kill()

In [3]:
ID = df.index.values
df = df.fillna('')
df = df.loc[:,~df.columns.duplicated()]
columns = list(df)

In [4]:
columnNumber = 0
dicts = {}
for column in df:
    i = 1
    thisDict = {}
    uniques = df[column].unique()
    for item in uniques:
        if not item in thisDict:
            if isinstance(item, numbers.Number):
                thisDict[item] = item
            else:
                if i in uniques:
                    j = i
                    while j not in uniques:
                        j = j + 1
                    thisDict[item] = j
                else:
                    thisDict[item] = i
                    i = i + 1
    dicts[columns[columnNumber]] =  thisDict
    columnNumber = columnNumber + 1
    
        

C:\Users\Luke Laptop\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if sys.path[0] == '':


In [5]:
df2 = df

In [6]:
i = 0
for item in dicts:
    if not is_numeric_dtype(df[item]):
        replacement = {item: dicts[item]}
        df2 = df2.replace(replacement)
    i = i + 1

In [7]:
for column in df2:
    df2[column] = df2[column].fillna(0)
    if not isinstance(df2[column][0],datetime.datetime):
        if df2[column].std() != 0:
            df2[column] = (df2[column] - df2[column].mean()) / df2[column].std()

In [8]:
tsne_em = TSNE(n_components=2, perplexity=38.0, n_iter=5000, verbose=1).fit_transform(df2)

[t-SNE] Computing 115 nearest neighbors...
[t-SNE] Indexed 675 samples in 0.001s...
[t-SNE] Computed neighbors for 675 samples in 0.050s...
[t-SNE] Computed conditional probabilities for sample 675 / 675
[t-SNE] Mean sigma: 1.501530
[t-SNE] KL divergence after 250 iterations with early exaggeration: 62.393234
[t-SNE] KL divergence after 2600 iterations: 0.555304


In [9]:
df_results = df
df_results['x'] = tsne_em[:,0]
df_results['y'] = tsne_em[:,1]
df_results['ID'] = df.index.values
for row in df_results:
    df_results[row] = df_results[row].astype(float, errors = 'ignore')
df_results['dob'] = df_results['dob'].astype(object)

In [10]:
df_results.drop(df_results.loc[df_results['DM']=='null'].index, inplace=True)

In [11]:
df_results = df_results.apply(pd.to_numeric, errors='ignore')

In [15]:
fig = px.scatter(df_results, x="x", y="y", color="DM Duration", symbol="DM", hover_data=["ID"]
                 ,category_orders={"DM Duration": df_results['DM Duration'].unique(),
                              "DM": df_results['DM'].unique()})
fig.update_traces(marker={'size': 7, 'line' : {'color' : 'rgba(0, 0, 0, 0.5)',
                                   'width' : 1}})

fig.layout.legend.y = 1.05
fig.layout.legend.x = 1.035
fig.layout.coloraxis.colorbar.y = 0.35

fig.show()

plotly.graph_objs._figure.Figure

In [13]:
df_results['DM Duration'].unique()

array([ 9497.,  8766., 11688.,  8401.,  1826., 13880., 10958.,  7305.,
        6209.,  9862., 20088., 12053.,  7670., 13514.,  3287.,  6940.,
        5479.,  5478., 14610.,  5844.,  4383.,  6575., 14245., 12784.,
       11323.,  5114., 10592.,  9131.,  4749.,  4018.,  9496., 13149.,
       15341., 14975.,  4748.,  8036., 18993., 10957., 23376.,  2922.,
       10227.,    nan, 23011., 16436.,  3653., 16802., 17532.,  7671.,
       12419.,  2557.,  8400., 30681.,  1096., 15706.,   730., 23741.,
        6574., 20089., 19723., 17897.,  2191., 18628., 19724., 19358.,
       20454., 12054., 16071., 18263., 12418., 24837., 20819.,  9132.,
       13879., 16437.,  2192.,  8035.,  6939., 14244.,  4017., 22280.,
       21185., 19359.,  3288., 17167.,  6210.,  1461.,  3652.])